<a href="https://colab.research.google.com/github/eun-88/DataScientist_07/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4_%EC%B2%98%EB%A6%AC_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98(inf).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📰 Reuters News Classification Project

본 프로젝트는 로이터 뉴스 데이터셋을 활용하여 단어장 크기(Vocab Size)에 따른 머신러닝 모델의 성능을 비교 분석합니다.
저는 **전체 단어(All words)**를 사용했을 때의 성능 측정을 담당하였습니다.

## 📊 실험 결과 (Vocab Size: None / All words)

| 모델명 (Model) | Accuracy | F1-Score (Weighted) | 비고 |
| :--- | :---: | :---: | :--- |
| **Linear SVC** | **0.8294** | **0.8236** | ⭐ **Best Performance** |
| **Voting** | 0.8294 | 0.8234 | |
| **SVM** | 0.8227 | 0.8122 | |
| **Logistic Regression** | 0.8130 | 0.8064 | |
| **XGBoost** | 0.8130 | 0.8061 | |
| **Dense (MLP)** | 0.8103 | 0.8016 | |
| **ComplementNB** | 0.7649 | 0.7346 | 속도 및 가성비 우수 |
| **Random Forest** | 0.7448 | 0.7180 | |
| **LightGBM** | 0.1923 | 0.1781 | 피처 과다로 인한 성능 저하 |

## 🧐 인사이트 요약
1. **최적의 모델**: 텍스트 분류 특성상 고차원 피처 처리에 능한 **Linear SVC**가 가장 높은 성능을 보였습니다.
2. **트리 모델의 한계**: 단어 수를 제한하지 않았을 때, LightGBM과 같은 트리 기반 모델은 오히려 성능이 급격히 저하되는 양상을 보였습니다.
3. **Voting 효율성**: 앙상블(Voting) 모델은 높은 성능을 보였으나, 단일 Linear SVC 모델과 큰 차이가 없어 효율성 측면에서는 단일 모델이 유리할 수 있습니다.

In [5]:
import pandas as pd
import numpy as np

# 1. 벡터화 도구
from sklearn.feature_extraction.text import TfidfVectorizer

# 2. 성능 평가 지표
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 3. 모델 리스트
from sklearn.linear_model import LogisticRegression    # Logistic Regression
from sklearn.svm import SVC                             # SVM
from sklearn.svm import LinearSVC                       # Linear SVC
from xgboost import XGBClassifier                       # XGBoost
from lightgbm import LGBMClassifier                     # LightGBM
from sklearn.ensemble import RandomForestClassifier     # Random Forest
from sklearn.naive_bayes import ComplementNB            # ComplementNB
from sklearn.ensemble import VotingClassifier           # Voting
from sklearn.neural_network import MLPClassifier       # Dense (MLP)

## 데이터 임포트 및 전처리

In [6]:
from tensorflow.keras.datasets import reuters

# 1. 데이터 로드 (은영님 담당: num_words=None)
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, test_split=0.2)

# 2. 단어 사전(index) 가져오기
word_index = reuters.get_word_index(path="reuters_word_index.json")

# 3. 인덱스 조정 (케라스 로이터 데이터 특성상 3을 더해줘야 실제 단어와 매칭됩니다)
index_to_word = {i+3: word for word, i in word_index.items()}
for i, word in enumerate(("<pad>", "<sos>", "<unk>")):
    index_to_word[i] = word

# 4. 숫자 시퀀스를 다시 텍스트 문장으로 변환
decoded_train = []
for sequence in x_train:
    decoded_train.append(' '.join([index_to_word[index] for index in sequence]))

decoded_test = []
for sequence in x_test:
    decoded_test.append(' '.join([index_to_word[index] for index in sequence]))

print(f"훈련용 데이터 개수: {len(decoded_train)}")
print(f"테스트용 데이터 개수: {len(decoded_test)}")

훈련용 데이터 개수: 8982
테스트용 데이터 개수: 2246


In [7]:
# 은영님 담당: 단어 수 제한 없음(None)
tfidf = TfidfVectorizer(max_features=None)

# 훈련 데이터 벡터화
tfidfv = tfidf.fit_transform(decoded_train)
# 테스트 데이터 벡터화
tfidfv_test = tfidf.transform(decoded_test)

print(f"TF-IDF 행렬의 크기 (피처 개수): {tfidfv.shape}")

TF-IDF 행렬의 크기 (피처 개수): (8982, 26506)


## 나이브베이즈

In [8]:
from sklearn.naive_bayes import ComplementNB

cb = ComplementNB()
cb.fit(tfidfv, y_train)
cb_predicted = cb.predict(tfidfv_test)

print("ComplementNB 정확도:", accuracy_score(y_test, cb_predicted))
print("ComplementNB F1-Score:", f1_score(y_test, cb_predicted, average='weighted'))

ComplementNB 정확도: 0.7649154051647373
ComplementNB F1-Score: 0.7346534179503126


##Linear SVC


In [9]:
from sklearn.svm import LinearSVC

lsvc = LinearSVC(C=1.0, penalty='l2', max_iter=500, random_state=0)
lsvc.fit(tfidfv, y_train)
lsvc_predicted = lsvc.predict(tfidfv_test)

print("Linear SVC 정확도:", accuracy_score(y_test, lsvc_predicted))
print("Linear SVC F1-Score:", f1_score(y_test, lsvc_predicted, average='weighted'))

Linear SVC 정확도: 0.8294746215494212
Linear SVC F1-Score: 0.823667985526927


##Logistic Regression

In [10]:
lr = LogisticRegression(C=100, penalty='l2', solver='liblinear', random_state=0)
lr.fit(tfidfv, y_train)
lr_predicted = lr.predict(tfidfv_test)

print("로지스틱 회귀 정확도:", accuracy_score(y_test, lr_predicted))
print("로지스틱 회귀 F1-Score:", f1_score(y_test, lr_predicted, average='weighted'))

로지스틱 회귀 정확도: 0.8130008904719501
로지스틱 회귀 F1-Score: 0.8064211282190699


##Random Forest

In [11]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
rf.fit(tfidfv, y_train)
rf_predicted = rf.predict(tfidfv_test)

print("랜덤 포레스트 정확도:", accuracy_score(y_test, rf_predicted))
print("랜덤 포레스트 F1-Score:", f1_score(y_test, rf_predicted, average='weighted'))

랜덤 포레스트 정확도: 0.744879786286732
랜덤 포레스트 F1-Score: 0.7180307918322248


##Dense (MLPClassifier)

In [12]:
from sklearn.neural_network import MLPClassifier

# 은영님은 피처가 많으므로 hidden_layer_sizes를 너무 크게 잡지 않는 게 팁입니다.
mlp = MLPClassifier(hidden_layer_sizes=(128,), max_iter=20, random_state=0)
mlp.fit(tfidfv, y_train)
mlp_predicted = mlp.predict(tfidfv_test)

print("Dense(MLP) 정확도:", accuracy_score(y_test, mlp_predicted))
print("Dense(MLP) F1-Score:", f1_score(y_test, mlp_predicted, average='weighted'))

Dense(MLP) 정확도: 0.8103294746215495
Dense(MLP) F1-Score: 0.8016688886533537


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


##LightGBM

In [ ]:
lgbm = LGBMClassifier(n_estimators=100, n_jobs=-1, random_state=0)
lgbm.fit(tfidfv.toarray(), y_train) # 일부 환경에선 .toarray()가 필요할 수 있음
lgbm_predicted = lgbm.predict(tfidfv_test.toarray())

print("LightGBM 정확도:", accuracy_score(y_test, lgbm_predicted))
print("LightGBM F1-Score:", f1_score(y_test, lgbm_predicted, average='weighted'))

##XGBoost

In [14]:
from xgboost import XGBClassifier

# n_jobs=-1로 CPU를 다 쓰도록 설정
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, tree_method='hist', random_state=0)
xgb.fit(tfidfv, y_train)
xgb_predicted = xgb.predict(tfidfv_test)

print("XGBoost 정확도:", accuracy_score(y_test, xgb_predicted))
print("XGBoost F1-Score:", f1_score(y_test, xgb_predicted, average='weighted'))

XGBoost 정확도: 0.8130008904719501
XGBoost F1-Score: 0.8061849600713158


##SVM (SVC)

In [15]:
from sklearn.svm import SVC

# kernel='linear'가 그나마 텍스트 분류에서 빠르고 정확합니다.
svm = SVC(kernel='linear', C=1.0, probability=True, random_state=0)
svm.fit(tfidfv, y_train)
svm_predicted = svm.predict(tfidfv_test)

print("SVM 정확도:", accuracy_score(y_test, svm_predicted))
print("SVM F1-Score:", f1_score(y_test, svm_predicted, average='weighted'))

SVM 정확도: 0.8227960819234195
SVM F1-Score: 0.8122819862949061


##Voting (Soft Voting)

In [17]:
# SVC로 대체하여 다시 정의
svm_for_voting = SVC(kernel='linear', C=1.0, probability=True, random_state=0)

voting_classifier = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('cb', cb),
        ('svm', svm_for_voting) # LinearSVC 대신 probability=True인 SVC 사용
    ],
    voting='soft'
)

voting_classifier.fit(tfidfv, y_train)
voting_predicted = voting_classifier.predict(tfidfv_test)

In [18]:
# 결과를 출력하는 코드입니다
from sklearn.metrics import accuracy_score, f1_score

acc = accuracy_score(y_test, voting_predicted)
f1 = f1_score(y_test, voting_predicted, average='weighted')

print("Voting 정확도:", acc)
print("Voting F1-Score:", f1)

Voting 정확도: 0.8294746215494212
Voting F1-Score: 0.8234093777582955


##ComplementNB

In [19]:
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score, f1_score

# 1. 모델 선언
cnb = ComplementNB()

# 2. 모델 학습 (은영님의 전체 단어 데이터 tfidfv 사용)
cnb.fit(tfidfv, y_train)

# 3. 예측
cnb_predicted = cnb.predict(tfidfv_test)

# 4. 결과 출력
print("ComplementNB 정확도:", accuracy_score(y_test, cnb_predicted))
print("ComplementNB F1-Score:", f1_score(y_test, cnb_predicted, average='weighted'))

ComplementNB 정확도: 0.7649154051647373
ComplementNB F1-Score: 0.7346534179503126



"텍스트에는 역시 선형 모델!"

Linear SVC가 가장 높은 점수를 기록했습니다. 텍스트 데이터는 피처가 아주 많은 고차원 데이터인데, 이런 경우 복잡한 트리보다 선형적으로 경계를 나누는 방식이 가장 잘 통한다는 게 증명되었네요.

"LightGBM의 대참사"

19%라는 점수는 거의 분류를 포기한 수준입니다. 이는 단어 수가 너무 많을 때(전체 단어 사용), 트리 모델이 정보의 바다에 빠져서 길을 잃었기 때문.

"Voting의 효율성"

Voting 점수가 최고점인 Linear SVC와 거의 비슷합니다. 즉, 여러 모델을 합치는 노력을 들인 것치고는 Linear SVC 단일 모델을 잘 쓰는 게 훨씬 효율적일 수 있다는 결론도 낼 수 있습니다.

.